In [97]:
import pandas as pd
import matplotlib.pyplot as plt

from references.state_mapping import STATE_DISTRICT_MAPPING

pd.set_option("display.max_columns", 50)

In [98]:
raw = pd.read_csv("data/processed/readable_transaction_01_09_2025.csv", na_values=["","-","   "])
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383584 entries, 0 to 383583
Data columns (total 13 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Property Type                    383584 non-null  object 
 1   District                         383584 non-null  object 
 2   Mukim                            383584 non-null  object 
 3   Scheme Name/Area                 383584 non-null  object 
 4   Road Name                        283469 non-null  object 
 5   Month, Year of Transaction Date  383584 non-null  object 
 6   Tenure                           383584 non-null  object 
 7   Land/Parcel Area                 383584 non-null  float64
 8   Unit                             383584 non-null  object 
 9   Main Floor Area                  283477 non-null  float64
 10  Unit                             283477 non-null  object 
 11  Unit Level                       383584 non-null  object 
 12  Tr

In [99]:
# Formatting/Cleaning Column Names
cols = raw.columns.str.strip().str.replace(" ","_").str.lower().str.replace(",","").str.replace("/","_or_")
print(cols)
data = raw.copy()
data.columns = cols
data.sample()

Index(['property_type', 'district', 'mukim', 'scheme_name_or_area',
       'road_name', 'month_year_of_transaction_date', 'tenure',
       'land_or_parcel_area', 'unit', 'main_floor_area', 'unit', 'unit_level',
       'transaction_price'],
      dtype='object')


,property_type,district,mukim,scheme_name_or_area,road_name,month_year_of_transaction_date,tenure,land_or_parcel_area,unit,main_floor_area,unit,unit_level,transaction_price
97032,1 - 1 1/2 Storey Terraced,Selama,Selama,TMN WAN ABDUL JAMAL (TMN PERMAI),LORONG WAN ABDUL JAMAL UTAMA,1/12/2021,Freehold,149.0,sq.m,74.0,sq.m,,170000


In [100]:
# Inspecting `unit` column 
print(data.iloc[:,8].unique())
print(data.iloc[:,10].unique())

# Dropping unit column as they are the same with the exception of missing data denoted by `-`
data = data.drop(columns="unit")
data.info()

['sq.m']
['sq.m' nan]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383584 entries, 0 to 383583
Data columns (total 11 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   property_type                   383584 non-null  object 
 1   district                        383584 non-null  object 
 2   mukim                           383584 non-null  object 
 3   scheme_name_or_area             383584 non-null  object 
 4   road_name                       283469 non-null  object 
 5   month_year_of_transaction_date  383584 non-null  object 
 6   tenure                          383584 non-null  object 
 7   land_or_parcel_area             383584 non-null  float64
 8   main_floor_area                 283477 non-null  float64
 9   unit_level                      383584 non-null  object 
 10  transaction_price               383584 non-null  int64  
dtypes: float64(2), int64(1), object(8)
memory usage: 32.2+ M

In [101]:
# Missing data analysis
print(data.isna().sum())
rows_missing = data[data.isna().any(axis=1)]
rows_missing.sample(5)

property_type                          0
district                               0
mukim                                  0
scheme_name_or_area                    0
road_name                         100115
month_year_of_transaction_date         0
tenure                                 0
land_or_parcel_area                    0
main_floor_area                   100107
unit_level                             0
transaction_price                      0
dtype: int64


,property_type,district,mukim,scheme_name_or_area,road_name,month_year_of_transaction_date,tenure,land_or_parcel_area,main_floor_area,unit_level,transaction_price
241308,Condominium/Apartment,Bahagian Kuching,Bahagian Kuching,MJC-COURTYARD SANCTUARY APARTMENT,NaN,1/8/2024,Leasehold,79.33,NaN,2,295000
325931,Flat,Kuala Lumpur,Mukim Ulu Kelang,PKNS ULU KELANG,NaN,1/8/2022,Leasehold,64.00,NaN,1,260000
280315,Condominium/Apartment,Melaka Tengah,Bdr Bkt Baru,PANGSAPURI SERI SIANTAN,NaN,1/7/2023,Freehold,70.00,NaN,1,115000
245297,Condominium/Apartment,Gombak,Bandar Hulu Kelang,UKAY BISTARI,NaN,1/6/2023,Freehold,78.97,NaN,14,340000
281461,Condominium/Apartment,Papar,Papar,LOK KAWI HEIGHTS,NaN,1/6/2023,Leasehold,82.22,NaN,4,240000


In [102]:
# The rows with missing data are typically multi level apartments, the roads are also likely shared. 
# The missing data likely will have minial impact on the analysis.
rows_missing.property_type.unique()

array(['1 - 1 1/2 Storey Terraced', 'Condominium/Apartment', 'Detached',
       'Flat', 'Low-Cost Flat', 'Town House'], dtype=object)

In [103]:
data["district"].unique()

array(['Alor Gajah', 'Bachok', 'Bagan Datuk', 'Bahagian Betong',
       'Bahagian Bintulu', 'Bahagian Kapit', 'Bahagian Kuching',
       'Bahagian Limbang', 'Bahagian Miri', 'Bahagian Mukah',
       'Bahagian Samarahan', 'Bahagian Sarikei', 'Bahagian Serian',
       'Bahagian Sibu', 'Bahagian Sri Aman', 'Baling', 'Bandar Baru',
       'Barat Daya', 'Batang Padang', 'Batu Pahat', 'Beaufort', 'Bentong',
       'Bera', 'Besut', 'Cameron Highland', 'Dungun', 'Gombak',
       'Gua Musang', 'Hilir Perak', 'Hulu Langat', 'Hulu Perak',
       'Hulu Selangor', 'Hulu Terengganu', 'Jasin', 'Jelebu', 'Jeli',
       'Jempol', 'Jerantut', 'Johor Bahru', 'Kampar', 'Kemaman',
       'Keningau', 'Kerian', 'Kinta', 'Klang', 'Kluang', 'Kota Bahru',
       'Kota Belud', 'Kota Kinabalu', 'Kota Marudu', 'Kota Setar',
       'Kota Tinggi', 'Kuala Kangsar', 'Kuala Krai', 'Kuala Langat',
       'Kuala Lumpur', 'Kuala Muda', 'Kuala Nerus', 'Kuala Pilah',
       'Kuala Selangor', 'Kuala Terengganu', 'Kuantan', '

In [104]:
# Renaming `RoMPin` in district column to `Rompin`
print(data.query("district == 'RoMPin'").head(2))
data["district"] = data["district"].replace("RoMPin", "Rompin")

         property_type district   mukim  scheme_name_or_area        road_name  \
376384  Low-Cost House   RoMPin   Bebar  PPR(M) DESA CEMPAKA     DESA CEMPAKA   
376385  Low-Cost House   RoMPin   Bebar  PPR(M) DESA CEMPAKA  JALAN CEMPAKA 1   

       month_year_of_transaction_date     tenure  land_or_parcel_area  \
376384                       1/5/2022  Leasehold                122.0   
376385                       1/5/2022  Leasehold                122.0   

        main_floor_area unit_level  transaction_price  
376384             63.0                        120000  
376385             63.0                        140000  


In [105]:
data.query("district == 'Rompin'").sample(5)

,property_type,district,mukim,scheme_name_or_area,road_name,month_year_of_transaction_date,tenure,land_or_parcel_area,main_floor_area,unit_level,transaction_price
17066,1 - 1 1/2 Storey Semi-Detached,Rompin,Bebar,BANDAR SATELITE MUADZAM SHAH,JALAN SETIABAKTI 6,1/6/2023,Leasehold,297.00,123.0,,350000
93234,1 - 1 1/2 Storey Terraced,Rompin,Keratong,PER GENERASI BAHARU FELDA KERATONG 4,JALAN KERATONG PERMAI 1,1/2/2024,Leasehold,162.58,77.0,,95000
17145,1 - 1 1/2 Storey Semi-Detached,Rompin,Bebar,"PRIMA MUADZAM SHAH,PINGGIR PELANGI,ROMPI",PRIMA MUADZAM SHAH,1/8/2024,Leasehold,260.00,94.0,,135000
93326,1 - 1 1/2 Storey Terraced,Rompin,Keratong,PER. GENERASI BAHARU FELDA SELANCAR 4/5,JALAN SELANCAR INDAH 4,1/10/2023,Leasehold,130.06,77.0,,90000
93191,1 - 1 1/2 Storey Terraced,Rompin,Endau,PERUMAHAN GEN BAHARU FELDA SELENDANG 2,JLN SELENDANG JAYA 5,1/8/2024,Leasehold,130.00,76.0,,90000


In [106]:
state = data["district"].map(STATE_DISTRICT_MAPPING)
data.insert(1,"state",state)
data.sample(5)

,property_type,state,district,mukim,scheme_name_or_area,road_name,month_year_of_transaction_date,tenure,land_or_parcel_area,main_floor_area,unit_level,transaction_price
145725,2 - 2 1/2 Storey Terraced,Johor,Johor Bahru,Plentong,MERIDIN EAST,MERIDIN EAST,1/5/2023,Freehold,108.70,157.0,,488000
310409,Detached,Kedah,Kuala Muda,Bandar Sungai Petani,ASTANA PARK HOMES - VILLA NATURA,VILLA NATURA,1/11/2023,Freehold,465.00,131.0,,590000
126998,2 - 2 1/2 Storey Terraced,Johor,Batu Pahat,Linau,TMN PUTERA INDAH,JALAN PUTERA INDAH,1/7/2023,Freehold,72.00,88.0,,80000
220523,2 - 2 1/2 Storey Terraced,Selangor,Sepang,Baru Baru Salak Tinggi,M SENYUM SALAK TINGGI,OFF JLN BBST,1/12/2023,Leasehold,111.48,126.0,,551000
95929,1 - 1 1/2 Storey Terraced,Pulau Pinang,Seberang Perai Utara,12,TAMAN SEPADU JAYA (D'HALONA PLACE),JALAN KUBANG MENERONG,1/12/2021,Freehold,133.00,92.0,,318000


In [107]:
# Persist the Results
data.to_csv("data/processed/processed_data.csv", index=False)